In [ ]:
!pip install syft==0.2.9

import torch
import pandas as pd
import numpy as np
import syft as sy
import copy
hook = sy.TorchHook(torch)
from torch import nn
import torch.nn.functional as F
from torch import optim
from sklearn.model_selection import train_test_split

In [ ]:
pip show syft

Name: syft
Version: 0.2.9
Summary: A Library for Private, Secure Deep Learning
Home-page: https://github.com/OpenMined/PySyft
Author: Andrew Trask
Author-email: contact@openmined.org
License: Apache-2.0
Location: /usr/local/lib/python3.7/dist-packages
Requires: scipy, importlib-resources, requests, tblib, RestrictedPython, tornado, aiortc, msgpack, websocket-client, openmined.threepio, psutil, shaloop, torch, syft-proto, notebook, websockets, requests-toolbelt, Pillow, numpy, dill, lz4, phe, torchvision, flask-socketio, Flask
Required-by: 


In [ ]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
Worker bob already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:secure_worker #objects:0>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataframe=np.genfromtxt("/content/drive/MyDrive/IIIT-K_Internship/Diabetes-FL/diabetes1.csv",delimiter=',',skip_header=1)

In [ ]:
dataframe

array([[  6.   , 148.   ,  72.   , ...,   0.627,  50.   ,   1.   ],
       [  1.   ,  85.   ,  66.   , ...,   0.351,  31.   ,   0.   ],
       [  8.   , 183.   ,  64.   , ...,   0.672,  32.   ,   1.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,   0.245,  30.   ,   0.   ],
       [  1.   , 126.   ,  60.   , ...,   0.349,  47.   ,   1.   ],
       [  1.   ,  93.   ,  70.   , ...,   0.315,  23.   ,   0.   ]])

In [ ]:
X=dataframe[:,0:8]
Y=dataframe[:,8]
Y=Y.reshape(768,1)
dtype=torch.float
device=torch.device("cpu")

In [ ]:
X

array([[  6.   , 148.   ,  72.   , ...,  33.6  ,   0.627,  50.   ],
       [  1.   ,  85.   ,  66.   , ...,  26.6  ,   0.351,  31.   ],
       [  8.   , 183.   ,  64.   , ...,  23.3  ,   0.672,  32.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,  26.2  ,   0.245,  30.   ],
       [  1.   , 126.   ,  60.   , ...,  30.1  ,   0.349,  47.   ],
       [  1.   ,  93.   ,  70.   , ...,  30.4  ,   0.315,  23.   ]])

In [ ]:
from sklearn.preprocessing import StandardScaler
model=StandardScaler()
X=model.fit_transform(X)


### Mobility dataset Import

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/IIIT-K_Internship/Mobility-prediction/data/vanet-trace-creteil-20130924-0700-0900.csv', sep=';')
df2 = pd.read_csv('/content/drive/MyDrive/IIIT-K_Internship/Mobility-prediction/data/vanet-trace-creteil-20130924-1700-1900.csv',sep=';')

In [ ]:
#dataframe=np.genfromtxt("/content/drive/MyDrive/IIIT-K_Internship/Mobility-prediction/data/vanet-trace-creteil-20130924-0700-0900.csv")



In [ ]:
df2.head()

,timestep_time,vehicle_slope,vehicle_lane,vehicle_angle,vehicle_type,vehicle_pos,vehicle_y,vehicle_x,vehicle_speed,vehicle_id
0,0.0,0.0,28925528_0,-44.63,Bus,6.1,722.81,1270.66,0.0,BusFlowNorthSouth0.0
1,0.0,0.0,239331354_0,160.22,Bus,6.1,471.48,1183.77,0.0,BusFlowSouthNorth0.0
2,0.0,0.0,-240120122_0,67.65,Bus,6.1,711.28,874.23,0.0,BusFlowWestEast0.0
3,0.0,0.0,-1274_0,16.81,Vehicle,4.1,517.45,1849.76,0.0,VehicleFlowEastToNorth.0
4,0.0,0.0,-1354_0,-93.35,Vehicle,4.1,420.70,1853.89,0.0,VehicleFlowEastToWest_0.0


In [ ]:
df1=df1[df1['vehicle_id']=='BusFlowNorthSouth0.0']

In [ ]:
cols = ['vehicle_slope', 'vehicle_lane', 'vehicle_type','vehicle_id', 'vehicle_y']# 'vehicle_angle','vehicle_pos', 'vehicle_speed'
df1.drop(cols, axis=1, inplace=True)

In [ ]:
df1=df1.to_numpy()

In [ ]:
df1

array([[ 0.00000e+00, -4.46300e+01,  6.10000e+00,  1.27066e+03,
         0.00000e+00],
       [ 1.00000e+00, -4.46300e+01,  7.65000e+00,  1.26956e+03,
         1.55000e+00],
       [ 2.00000e+00, -4.46300e+01,  1.20000e+01,  1.26650e+03,
         4.34000e+00],
       [ 3.00000e+00, -3.92700e+01,  1.90800e+01,  1.26445e+03,
         7.08000e+00],
       [ 4.00000e+00, -3.39000e+01,  2.78900e+01,  1.25956e+03,
         8.81000e+00],
       [ 5.00000e+00, -2.87400e+01,  3.92600e+01,  1.25372e+03,
         1.13700e+01],
       [ 6.00000e+00, -1.82200e+01,  5.29100e+01,  1.24947e+03,
         1.36500e+01],
       [ 7.00000e+00, -1.82200e+01,  6.63300e+01,  1.24529e+03,
         1.34200e+01],
       [ 8.00000e+00, -2.95400e+01,  7.95000e+01,  1.23941e+03,
         1.31700e+01],
       [ 9.00000e+00, -2.95400e+01,  8.83700e+01,  1.23505e+03,
         8.86000e+00],
       [ 1.00000e+01, -2.95400e+01,  9.40500e+01,  1.23226e+03,
         5.68000e+00],
       [ 1.10000e+01, -2.95400e+01,  9.5370

In [ ]:
#df1 = df1.iloc[:,1:2].values

In [ ]:
df1

,timestep_time,vehicle_x
1,0.0,1270.66
11,1.0,1269.56
21,2.0,1266.50
33,3.0,1264.45
45,4.0,1259.56
...,...,...
11409,175.0,1189.16
11535,176.0,1184.89
11661,177.0,1180.81
11787,178.0,1177.71


In [ ]:
len(df1)

180

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from torch.autograd import Variable

def sliding_windows(data, seq_length):
    x = []
    y = []

    for i in range(len(data)-seq_length-1):
        _x = data[i:(i+seq_length)]
        _y = data[i+seq_length]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)

sc = MinMaxScaler()
df1 = sc.fit_transform(df1)

seq_length = 4
x, y = sliding_windows(df1, seq_length)
'''
train_size = int(len(y) * 0.67)
test_size = len(y) - train_size

dataX = Variable(torch.Tensor(np.array(x)))
dataY = Variable(torch.Tensor(np.array(y)))

trainX = Variable(torch.Tensor(np.array(x[0:train_size])))
trainY = Variable(torch.Tensor(np.array(y[0:train_size])))

testX = Variable(torch.Tensor(np.array(x[train_size:len(x)])))
testY = Variable(torch.Tensor(np.array(y[train_size:len(y)])))
'''

'\ntrain_size = int(len(y) * 0.67)\ntest_size = len(y) - train_size\n\ndataX = Variable(torch.Tensor(np.array(x)))\ndataY = Variable(torch.Tensor(np.array(y)))\n\ntrainX = Variable(torch.Tensor(np.array(x[0:train_size])))\ntrainY = Variable(torch.Tensor(np.array(y[0:train_size])))\n\ntestX = Variable(torch.Tensor(np.array(x[train_size:len(x)])))\ntestY = Variable(torch.Tensor(np.array(y[train_size:len(y)])))\n'

In [ ]:
df_new = df1.to_numpy()

In [ ]:
df_new[0]

In [ ]:
X=df1[:,:4]
Y=df1[:,4]
#X=X.reshape(180,1)
Y=Y.reshape(180,1)
dtype=torch.float
device=torch.device("cpu")

In [ ]:
X

array([[ 0.00000e+00, -4.46300e+01,  6.10000e+00,  1.27066e+03],
       [ 1.00000e+00, -4.46300e+01,  7.65000e+00,  1.26956e+03],
       [ 2.00000e+00, -4.46300e+01,  1.20000e+01,  1.26650e+03],
       [ 3.00000e+00, -3.92700e+01,  1.90800e+01,  1.26445e+03],
       [ 4.00000e+00, -3.39000e+01,  2.78900e+01,  1.25956e+03],
       [ 5.00000e+00, -2.87400e+01,  3.92600e+01,  1.25372e+03],
       [ 6.00000e+00, -1.82200e+01,  5.29100e+01,  1.24947e+03],
       [ 7.00000e+00, -1.82200e+01,  6.63300e+01,  1.24529e+03],
       [ 8.00000e+00, -2.95400e+01,  7.95000e+01,  1.23941e+03],
       [ 9.00000e+00, -2.95400e+01,  8.83700e+01,  1.23505e+03],
       [ 1.00000e+01, -2.95400e+01,  9.40500e+01,  1.23226e+03],
       [ 1.10000e+01, -2.95400e+01,  9.53700e+01,  1.23162e+03],
       [ 1.20000e+01, -2.95400e+01,  9.55300e+01,  1.23154e+03],
       [ 1.30000e+01, -2.95400e+01,  9.55600e+01,  1.23152e+03],
       [ 1.40000e+01, -2.95400e+01,  9.55700e+01,  1.23152e+03],
       [ 1.50000e+01, -2.

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [ ]:

X_train =X[:int(len(df1)*0.8)]
y_train =Y[:int(len(df1)*0.8)]
X_test = X[int(len(df1)*0.8):]
y_test = Y[int(len(df1)*0.8):]

In [ ]:
len(y_train)

144

In [ ]:
'''
X_train= X_train.to_numpy()
y_train = y_train.to_numpy()
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()

In [ ]:

data = torch.from_numpy(X_train)
target = torch.from_numpy(y_train)
data,target=data.type(torch.FloatTensor),target.type(torch.FloatTensor)

'''
data = torch.from_numpy(trainX)
target = torch.from_numpy(trainY)
data,target=data.type(torch.FloatTensor),target.type(torch.FloatTensor)
'''

'\ndata = torch.from_numpy(trainX)\ntarget = torch.from_numpy(trainY)\ndata,target=data.type(torch.FloatTensor),target.type(torch.FloatTensor)\n'

In [ ]:
data.shape

torch.Size([144, 4])

In [ ]:
data_length,data_width=data.shape
partition=0.01 #Input only from 0 to 0.5 for bobs data. Alice is 1- partition

In [ ]:
bobs_data = data[0:int(data_length*partition)].send(bob)
bobs_target = target[0:int(data_length*partition)].send(bob)

alices_data = data[int(data_length*partition):].send(alice)
alices_target = target[int(data_length*partition):].send(alice)




In [ ]:
'''
bobs_data = data[0:int(data_length/2)].send(bob)
bobs_target = target[0:int(data_length/2)].send(bob)

alices_data = data[int(data_length/2):].send(alice)
alices_target = target[int(data_length/2):].send(alice)
'''

In [ ]:
(# Iniitalize A Toy Model
#model = nn.Linear(8,1)
#loss= torch.nn.MSELoss()

### Neural Network

In [ ]:
class my_network(torch.nn.Module):
    def __init__(self):
        super(my_network, self).__init__()
        self.fc1=nn.Linear(data_width,200)
        self.activ=nn.ReLU()
        self.fc2=nn.Linear(200,100)
        self.activ2=nn.ReLU()
        self.fc3=nn.Linear(100,1)
#         self.activ3=nn.ReLU()
    
    def forward(self,input_):
        a1=self.fc1(input_)
        a1=self.activ(a1)
        a2=self.fc2(a1)
        a2=self.activ2(a2)
        y=self.fc3(a2)
#         y=self.activ3(y)
        return y


In [ ]:
model = my_network()
model

my_network(
  (fc1): Linear(in_features=4, out_features=200, bias=True)
  (activ): ReLU()
  (fc2): Linear(in_features=200, out_features=100, bias=True)
  (activ2): ReLU()
  (fc3): Linear(in_features=100, out_features=1, bias=True)
)

In [ ]:
'''
class my_network(torch.nn.Module):
    def __init__(self):
        super(my_network, self).__init__()
        self.fc1=nn.Linear(data_width,200)
        self.activ=nn.ReLU()
        self.fc2=nn.Linear(200,100)
        self.activ2=nn.ReLU()
        self.fc3=nn.Linear(100,1)
#         self.activ3=nn.ReLU()
    
    def forward(self,input_):
        a1=self.fc1(input_)
        a1=self.activ(a1)
        a2=self.fc2(a1)
        a2=self.activ2(a2)
        y=self.fc3(a2)
#         y=self.activ3(y)
        return y

### LSTM

In [ ]:
'''
class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size)) #x.size(0)
        
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size)) #x.size(0)
        
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        
        h_out = h_out.view(-1, self.hidden_size)
        
        out = self.fc(h_out)
        
        return out

In [ ]:
#model=my_network()
'''
input_size = 1
hidden_size = 2
num_layers = 1

num_classes = 1

model=LSTM(num_classes, input_size, hidden_size, num_layers)
loss= torch.nn.MSELoss()


In [ ]:
loss= torch.nn.MSELoss()


In [ ]:
bobs_model = model.copy().send(bob)
alices_model = model.copy().send(alice)

bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.001)
alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.001)

In [ ]:
for i in range(1000):

    # Train Bob's Model
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = loss(bobs_pred,bobs_target)
    bobs_loss.backward()

    bobs_opt.step()
    bobs_loss = bobs_loss.get().data

    # Train Alice's Model
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = loss(alices_pred,alices_target)
    alices_loss.backward()

    alices_opt.step()
    alices_loss = alices_loss.get().data
    
#     print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

In [ ]:
iterations = 10
worker_iters = 500 # 200

for a_iter in range(iterations):
    
    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.001)
    alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.001)

    for wi in range(worker_iters):

        # Train Bob's Model
        bobs_opt.zero_grad()
        bobs_pred = bobs_model.forward(bobs_data)
        bobs_loss = loss(bobs_pred,bobs_target)
        bobs_loss.backward()        
        bobs_opt.step()
        bobs_loss = bobs_loss.get().data

        # Train Alice's Model
        alices_opt.zero_grad()
        alices_pred = alices_model.forward(alices_data)
        alices_loss = loss(alices_pred,alices_target)
        alices_loss.backward()
        alices_opt.step()
#         alices_opt.step(alices_data.shape[0])
        alices_loss = alices_loss.get().data
#         print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))
    
    alices_model.move(secure_worker)
    bobs_model.move(secure_worker)
    with torch.no_grad():
      model.fc1.weight.set_(((alices_model.fc1.weight.data *(1- partition) + bobs_model.fc1.weight.data * partition)).get())
      model.fc1.bias.set_(((alices_model.fc1.bias.data *(1- partition) + bobs_model.fc1.bias.data * partition)).get())
      model.fc2.weight.set_(((alices_model.fc2.weight.data *(1- partition) + bobs_model.fc2.weight.data * partition) ).get())
      model.fc2.bias.set_(((alices_model.fc2.bias.data *(1- partition) + bobs_model.fc2.bias.data * partition) ).get())
      model.fc3.weight.set_(((alices_model.fc3.weight.data *(1- partition) + bobs_model.fc3.weight.data * partition) ).get())
      model.fc3.bias.set_(((alices_model.fc3.bias.data *(1- partition) + bobs_model.fc3.bias.data * partition) ).get())

      #model.weight.set_(((alices_model.weight.data *(1- partition) + bobs_model.weight.data * partition) ).get())
      #model.bias.set_(((alices_model.bias.data *(1- partition) + bobs_model.bias.data * partition) ).get())
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(4.6439e+32) Alice:tensor(4.6790e+30)
Bob:tensor(7.5032e+29) Alice:tensor(7.5032e+29)
Bob:tensor(1.0134e+29) Alice:tensor(1.0134e+29)
Bob:tensor(1.3688e+28) Alice:tensor(1.3688e+28)
Bob:tensor(1.8487e+27) Alice:tensor(1.8487e+27)
Bob:tensor(2.4969e+26) Alice:tensor(2.4969e+26)
Bob:tensor(3.3725e+25) Alice:tensor(3.3725e+25)
Bob:tensor(4.5550e+24) Alice:tensor(4.5550e+24)
Bob:tensor(6.1522e+23) Alice:tensor(6.1522e+23)
Bob:tensor(8.3095e+22) Alice:tensor(8.3095e+22)


In [ ]:
X_test = torch.from_numpy(X_test)
y_test = torch.from_numpy(y_test)
X_test,y_test=X_test.type(torch.FloatTensor),y_test.type(torch.FloatTensor)
pred=model(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
MSE_error = mean_squared_error(y_test.detach().numpy(), pred.detach().numpy())
print('Testing Mean Squared Error is {}'.format(MSE_error))

Testing Mean Squared Error is 8.276279446737568e+22


In [ ]:
#-----------------

In [ ]:
pred=torch.abs(torch.round((pred)))
output=pred-y_test

In [ ]:
incorrect_labels=len(np.nonzero(output))

accuracy=1-(incorrect_labels/len(X_test))

accuracy

0.0

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test.detach().numpy(),pred.detach().numpy()))

              precision    recall  f1-score   support

         0.0       0.84      0.93      0.88       107
         1.0       0.78      0.60      0.67        47

    accuracy                           0.82       154
   macro avg       0.81      0.76      0.78       154
weighted avg       0.82      0.82      0.82       154

